# Lithium prescribing trends in England. 

## **Background and Rationale**
For over 60 years lithium has been the gold standard for the treatment and prevention of relapse in bipolar disorder.  In recent years, published literature has confirmed the superiority of lithium in this field, including two meta-analysis of RCTs (Severus et al. 2014, Miura et al. 2014) and one of observational studies (Kessing et al 2018).  Yet in various countries, authors have described a decline in lithium use (Karanti et al. 2016), including in the UK (Hayes et al. 2011, Lyall et al. 2019) 

We wish to describe long term lithium prescribing trends in England.  Previous authors have reported regional variations in lithium use across the country (POMH Lithium report, 2008). We would like to explore these national variations in use, to further understand trends in lithium prescribing in the UK. 

## **Proposed project**
### Title:  
Lithium prescribing trends
### Description:  
* Total number of items of lithium carbonate and lithium citrate dispensed per year from 1995 to 2018
* Total number of non-GP v GP items of lithium carbonate and lithium citrate dispensed per year
* Number of items of lithium carbonate per 1000 patient population for each CCG
* Number of items of lithium citrate per 1000 patient population for each CCG
* Number of items of non-GP v GP prescriptions for lithium carbonate for each region
* Number of items of non-GP v GP prescriptions for lithium citrate for each region



In [ ]:
#import libraries required for analysis
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from matplotlib.dates import DateFormatter
from ebmdatalab import bq
from ebmdatalab import charts
from ebmdatalab import maps
import matplotlib.gridspec as gridspec

In [ ]:
sql = """
WITH
bnf_tab AS (
SELECT
DISTINCT chemical,
chemical_code
FROM
ebmdatalab.hscic.bnf )
SELECT
rx.month,
rx.practice,
rx.pct,
SUBSTR(rx.bnf_code,1,9) AS chemical_code,
chemical,
sum(IF(rx.bnf_code LIKE "0402030K0%", items,0)) AS carbonate,
sum(IF(rx.bnf_code LIKE "0402030P0%", items,0)) AS citrate,
sum(items) AS total_lithium,
sum(actual_cost) AS total_cost
FROM
hscic.normalised_prescribing_standard AS rx
LEFT JOIN
bnf_tab
ON
chemical_code =SUBSTR(rx.bnf_code,1,9)
JOIN
hscic.practices AS prac
ON
rx.practice = prac.code 
JOIN
  hscic.ccgs AS ccgs
ON
rx.pct=ccgs.code
WHERE
  prac.setting = 4
  AND (bnf_code LIKE "0402030K0%" OR ##carbonate
  bnf_code LIKE "0402030P0%") ##citrate
  AND
ccgs.org_type='CCG'
GROUP BY
rx.month,
rx.practice,
rx.pct,
chemical_code,
chemical
ORDER BY
month"""

df_lithium = bq.cached_read(sql, csv_path='lithium.csv')

In [ ]:
df_lithium['month'] = df_lithium['month'].astype('datetime64[ns]')
df_lithium.head()

In [ ]:
df_lithium["chemical"].unique()#shows me the unique values within the chemical column

In [ ]:
df_lithium.nunique()

In [ ]:
df_carbonate = df_lithium.loc[df_lithium["chemical"] == "Lithium Carbonate"]
df_carbonate.head()#shows me the data for Lithium Carbonate

In [ ]:
df_carbonate.nunique()

In [ ]:
df_carbonate.describe()

In [ ]:
df_citrate = df_lithium.loc[df_lithium["chemical"] == "Lithium Citrate"]
df_citrate.head()#shows me the data for Lithium Citrate

In [ ]:
df_citrate.nunique()

In [ ]:
df_citrate.describe()

In [ ]:
ccg_lithium_totalcount = df_lithium.groupby(["pct", "month"], as_index=False).sum()
ccg_lithium_totalcount.head()#shows me the data grouped by PCT, therefore representing CCGs

In [ ]:
ccg_lithium_totalcount.describe()

### Display total no. of items dispensed per year

In [ ]:
plot1 = df_lithium.groupby("month").sum().plot(kind='line', title="Total number items of lithium")

In [ ]:
df_lithium.groupby("month")['total_lithium'].sum().plot(kind='line', title="Total number items of lithium")

In [ ]:
df_lithium.groupby("month")['carbonate'].sum().plot(kind='line', title='Total number of items of lithium carbonate')

In [ ]:
df_lithium.groupby("month")["citrate"].sum().plot(kind='line', title='Total number of items of lithium citrate')

In [ ]:
##get list size info
sql2 = """
SELECT month, 
pct_id AS pct,
AVG(total_list_size) AS list_size
FROM ebmdatalab.hscic.practice_statistics
group by 
month, pct
order by
month, pct,
list_size
"""
df_list = bq.cached_read(sql2, csv_path='df_list.csv')
df_list['month'] = df_list['month'].astype('datetime64[ns]')
df_list.head(5)

In [ ]:
ccg_lithium_count = df_lithium.groupby(["month", "pct"])["total_lithium"].sum().reset_index()
ccg_lithium_count.head()

In [ ]:
#merge dataframes to show items per 1000

df_lithium_per1000 = pd.merge(ccg_lithium_count, df_list, on=['month', 'pct'])
df_lithium_per1000['items_per_1000'] = round(1000* (df_lithium_per1000['total_lithium']/df_lithium_per1000['list_size']),0)
df_lithium_per1000.head()

In [ ]:
df_lithium_per1000["month"] = pd.to_datetime(df_lithium_per1000['month']).dt.date

In [ ]:
df_lithium_per1000.describe()

In [ ]:
#create sample deciles

charts.deciles_chart(
        df_lithium_per1000,
        period_column='month',
        column='total_lithium',
        title="Number of lithium items per 1000 patient",
        ylabel='Items per 1000',
        show_outer_percentiles=True,
        show_legend=False,
)

plt.show()

### Describe lithium items prescribed by non-GP facilities in UK

In [ ]:
sql3 = """
WITH
bnf_tab AS (
SELECT
DISTINCT chemical,
chemical_code
FROM
ebmdatalab.hscic.bnf )
SELECT
rx.month,
rx.practice,
prac.setting,
SUBSTR(rx.bnf_code,1,9) AS chemical_code,
chemical,
sum(IF(rx.bnf_code LIKE "0402030K0%", items,0)) AS carbonate,
sum(IF(rx.bnf_code LIKE "0402030P0%", items,0)) AS citrate,
sum(items) AS total_lithium,
sum(actual_cost) AS total_cost
FROM
hscic.normalised_prescribing_standard AS rx
LEFT JOIN
bnf_tab
ON
chemical_code =SUBSTR(rx.bnf_code,1,9)
JOIN
hscic.practices AS prac
ON
rx.practice = prac.code 

WHERE
  prac.setting !=4
  AND (bnf_code LIKE "0402030K0%" OR ##carbonate
  bnf_code LIKE "0402030P0%") ##citrate
  
GROUP BY
rx.month,
rx.practice,
rx.pct,
prac.setting,
chemical_code,
chemical
ORDER BY
month"""

df_nonGPlithium = bq.cached_read(sql3, csv_path='nonGPlithium.csv')

In [ ]:
df_nonGPlithium['month'] = df_nonGPlithium['month'].astype('datetime64[ns]')
df_nonGPlithium

In [ ]:
df_nonGPlithium.groupby('month').sum()

In [ ]:
headers = [
    'code','name','postcode','status_code','ccg','setting']
nonGPbreakdown = pd.read_csv("lithium.csv", names=headers, usecols=[0,1,9,12,23,25])
nonGPbreakdown.head()

In [ ]:
type(plot1)